---
---

<h1><center><ins>Numerical Methods: Exam 1</ins></center></h1>
<h2><center>November 12, 09:45-11:15 </center></h2>

Please compute the below questions in the python notebook, *documenting your code* where needed. 

Remember also to **_answer any discussion points asked in the question_**.  

***Before 11:20*** your notebook needs to be uploaded to Moodle, and/or emailed to (both) Kristina and Alina <br> (kristina.kislyakova@univie.ac.at, alina.boecker@univie.ac.at)<br>
<br>
This means you have roughly 60 minutes to complete the exercises in this part.

You may use the lecture notes and any of your exercise notebooks freely, but you can **not** use ChatGPT code assistant or other AI coding tools.<br> 
**_Collaboration or plagarized code in any way is prohibited and no phones or communication will be allowed_**.

---
---

In [ ]:
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv, lu_solve, lu_factor
from scipy.interpolate import CubicSpline

## Exercise 1 (9 Points)

Consider the system:

$$
\begin{cases}
2x_1 + 2x_2 + 2x_3 = 6 \\
2.001x_1  + 2x_2 + 2x_3 = 6.001 \\
2x_1  + 2.001x_2 + 2x_3 = 6.001
\end{cases}
$$

**(a)** Write the system in matrix form $A\mathbf{x} = \mathbf{b}$. Solve for $\mathbf{x}$ by calculating the inverse of $A$ (Hint: under https://numpy.org/doc/stable/reference/routines.linalg.html#solving-equations-and-inverting-matrices you should find the build-in function you are looking for). (1 Point)

**(b)** Perform LU decomposition to find the solution vector $\mathbf{x}$. Calculate the difference between the exact solution $\mathbf{b}$ and your numerical one. Is it the same for the two methods? What does this imply for possibly even larger and thus more complex matrices? (1.5 Points)

Now consider the function

$$y = f(x) = \frac{1}{1+x^2}$$

**(c)** Plot the function on a domain of [-10,10] in x. Calculate the numerical derivative of the function using the forward finite difference and plot the result next to it (Hint: use a subplot). Choose the step-size $h$ such that the truncation error is 1e-2 at $x=1$. (Hint: the truncation error is $\epsilon\sim\frac{|f^{\prime\prime}(x)|}{2}h$). (3 Points)

**(d)** Now calculate the second derivative of y numerically. Use the same step-size as before as well as one that is 10 times larger. Using the samples of x with step-size of $10h$, interpolate y using a cubic spline (from scipy.interpolate.CubicSpline: https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html#scipy.interpolate.CubicSpline). Use the CubicSpline to find the second derivative using step-size $10h$ (Hint: look at the first example under the link). Compare all three numerical derivatives in a figure. What is your conclusion? (3.5 Points)

**(e) Extra Point**  Now add a random error of 1\% to your values of y evaluated with step-size $h$ (Hint: use np.random.normal(y,0.01)). Calculate again the first numerical derivative. Which percentage level is the difference between the new derivative values and the ones from **(c)**? Comment on what is happening. (1 Point)

In [31]:
#a
import numpy as np

a = np.array([[2., 2., 2.], [2.0001, 2., 2.], [2., 2.0001, 2.]])
b = np.array([[6], [6.0001], [6.0001]])
inv(a)
a @ inv(a)
print (a @ inv(a))
#test
x = np.linalg.solve(a, b)
print(x)


[[ 1.00000000e+00 -3.63797881e-12  0.00000000e+00]
 [ 7.27595761e-12  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -3.63797881e-12  1.00000000e+00]]
[[1.]
 [1.]
 [1.]]


In [25]:
#b
import numpy as np
from scipy.linalg import lu


A = np.array([[2., 2., 2.], [2.0001, 2., 2.], [2., 2.0001, 2.]])
b = np.array([[6], [6.0001], [6.0001]])
P, L, U = lu(A)   # A = P @ L @ U
print("\nPermutation matrix P:\n", P)
print("\nLower triangular matrix L:\n", L)
print("\nUpper triangular matrix U:\n", U)

# Forward substitution L * y = P * b
Pb = P @ b
y = np.zeros_like(b)
for i in range(len(b)):
    y[i] = Pb[i] - np.dot(L[i, :i], y[:i])

# Backward substitution U * x = y
x = np.zeros_like(b)
for i in reversed(range(len(b))):
    x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]


print("\nSolution x = [x1, x2, x3]:\n", x)



Permutation matrix P:
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]

Lower triangular matrix L:
 [[1.        0.        0.       ]
 [0.99995   1.        0.       ]
 [0.99995   0.4999875 1.       ]]

Upper triangular matrix U:
 [[2.00010000e+00 2.00000000e+00 2.00000000e+00]
 [0.00000000e+00 1.99995000e-04 9.99950003e-05]
 [0.00000000e+00 0.00000000e+00 4.99987500e-05]]

Solution x = [x1, x2, x3]:
 [[-9.44308489e-12]
 [-1.00000000e+00]
 [ 4.00005000e+00]]


b 
I get two different solutions for the two methods. That means if the matrixes get larger the answer should get more unprecise

In [ ]:
#c 
from scipy.integrate import quad
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import inv, lu_solve, lu_factor
from scipy.interpolate import CubicSpline

def y(x):
    return 1/(1+x**2)



## Exercise 2 (9 Points)

Consider the following two curves:
$$ f(x) = 6 - (x+2)^2 $$
$$ g(x) = \frac{1}{2}x $$

**(a)** Use _**your own version**_ of the **Newton Raphson method** to find the two intersections of these two curves (Hint: You can reformulate the problem as finding the roots of a single function built from the two given ones.). Plot the two curves in a figure to understand sensible guesses for the two starting points. Also plot the two two intersection points you found with the Newton Raphson method. (Hint: Use xlims=[-10,10] and ylims=[-10,10] for the plot). (2 Points)

**(b)** Use _**your own version**_ of the **Trapezoid method** to find the area between the two curves (Hint: use the points calculated from **(a)** as the upper and lower limits for the integration). Sample the integrand at 100 equally spaced abscissae. Compare this result with the **_build-in version_** of **scipy.integrate.quad**, which uses adaptive Gaussian quadrature. (3 Points)

**(c)** How many samples of abscissae do you need for your Trapezoid method in order to reach the same result provided by scipy.integrate.quad with a precision of 1e-4? Per iteration multiply the number of abscissae by 10. How does this compare to the error returned by the scipy.integrate.quad routine? Comment on this. (3 Points)

**(d)** In your figure from **(a)** shade the area you calculated between the two curves. Hint: Use fill_between from matplotlib. (1 Point)